# Initialize cluster and Moving randomly any molecule
# avoiding that any atoms overlaps each other:

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

# xyz_view.show()

In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

# importing de Molecule Class
from amcess.base_molecule import Molecule, Cluster

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [4]:
hf = {"atoms": [("H", 0, 0, 0), ("F", 1, 0, 0)]}
cc = {"atoms": [("C", 0, 0, 0), ("C", 1.2, 0, 0)]}
ab = Cluster(hf, cc)


In [17]:
xyz_data = ab.xyz.replace('\t', '').split('\n')
xyz_data = xyz_data[2:]
xyz_data = ';'.join(xyz_data)

print(xyz_data)

H          0.00000000     0.00000000     0.00000000;F          1.00000000     0.00000000     0.00000000;C          0.00000000     0.00000000     0.00000000;C          1.20000000     0.00000000     0.00000000;


In [ ]:
ab_xyz = ""

ab.sphere_radius = 4
# ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.random_generator.choice(ab.total_molecules)

    ab = ab.move_molecule(
        molecule=mol,
        max_step=3,
        max_rotation=10,
        max_closeness=0.5,
    )
    # print(ab.xyz)
    ab_xyz += ab.xyz

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

In [ ]:
view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.sphere_radius * 1.2,
    sphere_center = ab.sphere_center,
)
view.show()


In [ ]:
ab_xyz = ""
ab.sphere_radius = 4
ab.freeze_molecule = None


In [ ]:
ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.random_generator.choice(ab.total_molecules)

    ab = ab.move_molecule(
        molecule=mol,
        max_step=4.0,
        max_rotation=30,
        max_closeness=1.0,
    )
    # print(ab.xyz)
    ab_xyz += ab.xyz

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

# print(ab.freeze_molecule)
# print(ab.cluster_dictionary)

view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.sphere_radius * 1.2,
    sphere_center = ab.sphere_center,
)
view.show()

In [ ]:
import numpy as np

def overlapping(
    first_coordinates: list,
    second_coordinates: list,
    max_closeness: float = 1.0,
) -> bool:
    for first_atom in first_coordinates:
        for second_atom in second_coordinates:
            distance = np.linalg.norm(
                np.asarray(first_atom) - np.asarray(second_atom)
            )

            if distance < max_closeness:
                return True

    return False

In [ ]:

def initialize_cluster(mol, max_closeness: float = 1.0) -> object:
    # center of mass coordinates
    sc_x = mol.sphere_center[0]
    sc_y = mol.sphere_center[1]
    sc_z = mol.sphere_center[2]

    # initializing a new cluster moving the first molecule
    # to the center of the cluster sphere
    molecule = mol.get_molecule(0)
    new_cluster = molecule.translate(0, sc_x, sc_y, sc_z)

    for i in range(1, mol.total_molecules):
        # moving the next single molecule into the cluster sphere
        molecule = mol.get_molecule(i).translate(0, sc_x, sc_y, sc_z)

        if overlapping(molecule.coordinates, new_cluster.coordinates):
            new_cluster += molecule
            new_cluster = new_cluster.move_molecule(
                i,
                max_step=None,
                max_rotation=None,
                max_closeness=max_closeness,
            )
        else:
            new_cluster += molecule

    return Cluster(
        new_cluster,
        freeze_molecule=mol.freeze_molecule,
        sphere_radius=mol.sphere_radius,
        sphere_center=mol.sphere_center,
    )

In [ ]:
hf6 = 6 * Cluster(hf)

hf6.sphere_radius = 4
hf6.sphere_center = 10, 10, 10

print(hf6.seed)

In [ ]:
hf6.seed = 932570

new_hf6 = initialize_cluster(hf6, max_closeness=4.0)
print(new_hf6.xyz)

In [ ]:
new_view = visualize(
    xyz_file = new_hf6.xyz,
    sphere_radius = new_hf6.sphere_radius * 1.2,
    sphere_center = new_hf6.sphere_center,
)
new_view.show()

In [ ]:
water_molecule=[
    ("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)
    ]

w2 = Cluster(water_molecule, water_molecule)

# the water dimer was initialized with the same water molecule, so
# both molecules has the same coordinates an those atoms overlap each other
print(w2.xyz)

In [ ]:
#let's move any molecule randomly to avoid averlaping
new_w2 = w2.move_molecule(
    molecule = 1,
    max_step = 1.4,
    max_rotation = 30,
    max_closeness = 1.2,
)
print(new_w2.xyz)

In [ ]:
new_w2 = w2.move_molecule(1)
print(new_w2.xyz)

In [ ]:
from copy import deepcopy
from numpy import random
random_gen = random.default_rng(1234)

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# w2.freeze_molecule = 0

w2.sphere_radius = 4

new_w2 = deepcopy(w2)
new_w2 = new_w2.initialize_cluster(max_closeness=1)

# to save snapshot and show as a movie
w2_xyz = ""

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_w2.total_molecules)

    print(new_w2.xyz)


    # saving coordinates as a string
    w2_xyz += new_w2.xyz
    # if i % 10 == 0: 
    #     w2_xyz += new_w2.xyz

    #----------------------------------------------------------

    new_w2 = new_w2.move_molecule(
        molecule=mol,
        max_step=2,
        max_rotation=30,
        max_closeness=0.5,
    )

# # file_name = "w2.xyz"
# # with open(file_name, "w") as f:
# #     f.write(w2_xyz)

In [ ]:
new_view = visualize(
    xyz_file = w2_xyz,
    sphere_radius =w2.sphere_radius * 1.2,
    sphere_center = w2.sphere_center,
)
new_view.show()

In [ ]:
# for several molecules we could move randomly one-by-one
# or we could just `initialize` this cluster with all its molecules 
# moved around avoiding overlapping
w6 = 3 * w2

# let's define the spherical boundary conditions
w6.sphere_radius = 10

# maximum closeness is the minimum distance for two atoms
new_w6 = w6.initialize_cluster(max_closeness=3)

print(new_w6.xyz)

!pip install py3Dmol
import py3Dmol

In [ ]:
new_view = visualize(
    xyz_file = new_w6.xyz,
    sphere_radius =new_w6.sphere_radius * 1.2,
    sphere_center = new_w6.sphere_center,
)
new_view.show()


In [ ]:
from data.molecules_coordinates import water, ibuprofen 

# let's create a molecular object for the ibuprofen
ibu = Molecule(ibuprofen["atoms"])

# let's create a molecular object for one water molecule
w = Molecule(water["atoms"])

In [ ]:
w6_overlap = Cluster(6 * w)

# let's initialize a cluster of six water without overlapping
w6 = Cluster(6 * w).initialize_cluster(max_closeness=2)


print(w6.xyz)


In [ ]:
new_view = visualize(
    xyz_file = w6.xyz,
    sphere_radius = 5,
    sphere_center = w6.sphere_center,
)
new_view.show()

In [ ]:
ibu_w6 = Cluster(ibu + w6)

# let's define a sphere with center around the carboxylic acid group
ibu_w6.sphere_center = -3, 0.2, 0.2
ibu_w6.sphere_radius = 4

ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

print(ibu_w6.xyz)

In [ ]:
new_view = visualize(
    xyz_file = ibu_w6.xyz,
    sphere_radius = ibu_w6.sphere_radius * 1.2,
    sphere_center = ibu_w6.sphere_center,
    style='stick',
)
new_view.show()

In [ ]:
from numpy import random
random_gen = random.default_rng(1234)

new_ibu_w6 = deepcopy(ibu_w6)

# to save snapshot and show as a movie
new_ibu_w6_xyz = ""

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# new_ibu_w6.freeze_molecule = 0

new_ibu_w6.sphere_radius = 3 
new_ibu_w6.sphere_center = -6.12410200, 0.37976300, 0.38520700


# to avoid overlapping and any molecule out of the sphere
ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_ibu_w6.total_molecules)

    if mol == 0:
        continue

    # saving coordinates as a string
    new_ibu_w6_xyz += new_ibu_w6.xyz
    # if i % 10 == 0: 
        # new_ibu_w6_xyz += new_ibu_w6.xyz

    # #----------------------------------------------------------

    new_ibu_w6 = new_ibu_w6.move_molecule(
        molecule=mol,
        max_closeness=0.5,
        max_step=2,
        max_rotation=30,    
    )

    #----------------------------------------------------------


    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )
# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
# print(new_ibu_w6_xyz)

# open("new_ibu_w6.xyz", "w").write(new_ibu_w6_xyz)

In [ ]:
new_view = visualize(
    xyz_file = new_ibu_w6_xyz,
    sphere_radius = new_ibu_w6.sphere_radius * 1.2,
    sphere_center = new_ibu_w6.sphere_center,
    style='stick',
)
new_view.show()